# 🔒 Security Copilot - Demo

Este notebook demonstra como usar o **Security Copilot** para análise de segurança.

## Funcionalidades

- 🔍 Scan de vulnerabilidades (OWASP Top 10)
- 💉 Detecção de SQL Injection, XSS, etc.
- 🔐 Análise de autenticação e autorização
- 📊 Verificação de dependências vulneráveis
- 🛡️ Recomendações de correção


## 1. Setup Inicial


In [1]:
# Configuração do ambiente
import sys
sys.path.insert(0, '..')

from src.utils import setup_logging
setup_logging(level="INFO")

print("✅ Setup concluído!")


✅ Setup concluído!


In [2]:
# Importar o Security Copilot
from src.copilots import SecurityCopilot

# Inicializar o copiloto
security = SecurityCopilot()

print(f"🔒 Security Copilot inicializado!")
print(f"   Modelo: {security.model_config.name}")
print(f"   Provider: {security.model_config.provider}")


[12/11/25 15:55:26] INFO     2025-12-11 15:55:26,940 - src.agents.base - INFO - Copiloto 'Security Analyst'        
                             inicializado com openai/gpt-4o

🔒 Security Copilot inicializado!
   Modelo: gpt-4o
   Provider: openai


## 2. Detectar SQL Injection

Vamos analisar código vulnerável a SQL Injection.


In [3]:
# ⚠️ Código VULNERÁVEL - NÃO USE EM PRODUÇÃO!
codigo_vulneravel_sql = '''
import sqlite3

def get_user(user_id):
    """Busca usuário pelo ID - VULNERÁVEL!"""
    conn = sqlite3.connect("database.db")
    cursor = conn.cursor()
    
    # ❌ SQL Injection - concatenação direta
    query = f"SELECT * FROM users WHERE id = {user_id}"
    cursor.execute(query)
    
    return cursor.fetchone()

def login(username, password):
    """Faz login - VULNERÁVEL!"""
    conn = sqlite3.connect("database.db")
    cursor = conn.cursor()
    
    # ❌ SQL Injection - f-string com input do usuário
    query = f"SELECT * FROM users WHERE username = '{username}' AND password = '{password}'"
    cursor.execute(query)
    
    user = cursor.fetchone()
    if user:
        return {"status": "success", "user": user}
    return {"status": "failed"}

def search_products(term):
    """Busca produtos - VULNERÁVEL!"""
    conn = sqlite3.connect("database.db")
    cursor = conn.cursor()
    
    # ❌ SQL Injection
    query = "SELECT * FROM products WHERE name LIKE '%" + term + "%'"
    cursor.execute(query)
    
    return cursor.fetchall()
'''

print("⚠️ Código com vulnerabilidades SQL Injection:")
print(codigo_vulneravel_sql)


⚠️ Código com vulnerabilidades SQL Injection:

import sqlite3

def get_user(user_id):
    """Busca usuário pelo ID - VULNERÁVEL!"""
    conn = sqlite3.connect("database.db")
    cursor = conn.cursor()

    # ❌ SQL Injection - concatenação direta
    query = f"SELECT * FROM users WHERE id = {user_id}"
    cursor.execute(query)

    return cursor.fetchone()

def login(username, password):
    """Faz login - VULNERÁVEL!"""
    conn = sqlite3.connect("database.db")
    cursor = conn.cursor()

    # ❌ SQL Injection - f-string com input do usuário
    query = f"SELECT * FROM users WHERE username = '{username}' AND password = '{password}'"
    cursor.execute(query)

    user = cursor.fetchone()
    if user:
        return {"status": "success", "user": user}
    return {"status": "failed"}

def search_products(term):
    """Busca produtos - VULNERÁVEL!"""
    conn = sqlite3.connect("database.db")
    cursor = conn.cursor()

    # ❌ SQL Injection
    query = "SELECT * FROM products WHERE name L

In [4]:
# Analisar vulnerabilidades
print("🔍 Analisando código...\n")

resultado = security.vulnerability_scan(
    code=codigo_vulneravel_sql,
    language="python"
)

if resultado.success:
    print("🔒 Análise de Segurança:\n")
    print(resultado.content)
else:
    print(f"❌ Erro: {resultado.content}")


🔍 Analisando código...

🔒 Análise de Segurança:

## Vulnerabilidades Encontradas

### [Critical] A03:2021 - Injection
- **Localização**: `get_user` função, linha 8
- **Descrição**: SQL Injection devido à concatenação direta de entrada do usuário na query SQL.
- **Impacto**: Um atacante pode manipular a entrada `user_id` para executar comandos SQL arbitrários, potencialmente acessando ou modificando dados sensíveis.
- **CWE**: CWE-89: Improper Neutralization of Special Elements used in an SQL Command ('SQL Injection')
- **Recomendação**: Use consultas parametrizadas para evitar a injeção de SQL.
- **Código Corrigido**:
    ```python
    def get_user(user_id):
        """Busca usuário pelo ID - Corrigido"""
        conn = sqlite3.connect("database.db")
        cursor = conn.cursor()

        # ✅ Consulta parametrizada
        query = "SELECT * FROM users WHERE id = ?"
        cursor.execute(query, (user_id,))

        return cursor.fetchone()
    ```

### [Critical] A03:2021 - Injection


## 3. Detectar XSS e Problemas de Autenticação

Código com múltiplas vulnerabilidades.


In [5]:
# ⚠️ Código com XSS e problemas de autenticação
codigo_vulneravel_web = '''
from flask import Flask, request, render_template_string
import hashlib

app = Flask(__name__)

# ❌ Senha hardcoded
ADMIN_PASSWORD = "admin123"
SECRET_KEY = "minha-chave-secreta-123"

@app.route("/search")
def search():
    query = request.args.get("q", "")
    
    # ❌ XSS - renderiza input do usuário diretamente
    html = f"<h1>Resultados para: {query}</h1>"
    return render_template_string(html)

@app.route("/profile")
def profile():
    name = request.args.get("name", "")
    
    # ❌ XSS refletido
    return f"<html><body>Olá, {name}!</body></html>"

@app.route("/login", methods=["POST"])
def login():
    username = request.form.get("username")
    password = request.form.get("password")
    
    # ❌ Hash fraco (MD5)
    password_hash = hashlib.md5(password.encode()).hexdigest()
    
    # ❌ Comparação insegura
    if password == ADMIN_PASSWORD:
        return "Login OK"
    
    return "Falha no login"

@app.route("/api/data")
def get_data():
    # ❌ Sem autenticação
    # ❌ Expõe dados sensíveis
    return {
        "users": [...],
        "passwords": [...],
        "api_keys": [...]
    }

@app.route("/debug")
def debug():
    # ❌ Debug habilitado em produção
    import os
    return str(os.environ)
'''

print("⚠️ Código com XSS e problemas de autenticação:")
print(codigo_vulneravel_web)


⚠️ Código com XSS e problemas de autenticação:

from flask import Flask, request, render_template_string
import hashlib

app = Flask(__name__)

# ❌ Senha hardcoded
ADMIN_PASSWORD = "admin123"
SECRET_KEY = "minha-chave-secreta-123"

@app.route("/search")
def search():
    query = request.args.get("q", "")

    # ❌ XSS - renderiza input do usuário diretamente
    html = f"<h1>Resultados para: {query}</h1>"
    return render_template_string(html)

@app.route("/profile")
def profile():
    name = request.args.get("name", "")

    # ❌ XSS refletido
    return f"<html><body>Olá, {name}!</body></html>"

@app.route("/login", methods=["POST"])
def login():
    username = request.form.get("username")
    password = request.form.get("password")

    # ❌ Hash fraco (MD5)
    password_hash = hashlib.md5(password.encode()).hexdigest()

    # ❌ Comparação insegura
    if password == ADMIN_PASSWORD:
        return "Login OK"

    return "Falha no login"

@app.route("/api/data")
def get_data():
    # ❌

In [6]:
# Analisar vulnerabilidades web
print("🔍 Analisando código web...\n")

resultado = security.vulnerability_scan(
    code=codigo_vulneravel_web,
    language="python"
)

if resultado.success:
    print("🔒 Análise de Segurança:\n")
    print(resultado.content)
else:
    print(f"❌ Erro: {resultado.content}")


🔍 Analisando código web...

🔒 Análise de Segurança:

## Vulnerabilidades Encontradas

### [Critical] A01:2021 - Broken Access Control
- **Localização**: Linha 30
- **Descrição**: A rota `/api/data` expõe dados sensíveis sem qualquer autenticação ou controle de acesso.
- **Impacto**: Um atacante pode acessar informações confidenciais, como usuários, senhas e chaves de API.
- **CWE**: CWE-284: Improper Access Control
- **Recomendação**: Implementar autenticação e autorização adequadas para proteger o acesso a dados sensíveis.
- **Código Corrigido**:
    ```python
    from flask import jsonify, abort

    @app.route("/api/data")
    def get_data():
        if not user_is_authenticated():
            abort(403)
        return jsonify({
            "users": [...],
            "passwords": [...],
            "api_keys": [...]
        })
    ```

### [Critical] A02:2021 - Cryptographic Failures
- **Localização**: Linha 21
- **Descrição**: Uso de algoritmo de hash fraco (MD5) para senhas.
- **

## 4. Testar via API (Docker)

Se o Docker estiver rodando, podemos testar via API HTTP.


In [7]:
import httpx

API_URL = "http://localhost:8000"

# Verificar se API está disponível
try:
    response = httpx.get(f"{API_URL}/health", timeout=5)
    print(f"✅ API disponível: {response.json()}")
    API_DISPONIVEL = True
except:
    print("⚠️ API não está rodando. Execute: make docker-up")
    API_DISPONIVEL = False


✅ API disponível: {'status': 'healthy'}


In [8]:
# Análise de segurança via API
if API_DISPONIVEL:
    codigo_api = '''
import os
import subprocess

def execute_command(cmd):
    # ❌ Command Injection
    os.system(cmd)

def run_script(script_name):
    # ❌ Command Injection
    subprocess.call(f"python {script_name}", shell=True)

def read_file(filename):
    # ❌ Path Traversal
    with open(f"/data/{filename}") as f:
        return f.read()
'''
    
    print("🔍 Analisando via API...\n")
    
    response = httpx.post(
        f"{API_URL}/api/security",
        json={
            "code": codigo_api,
            "language": "python"
        },
        timeout=120
    )
    
    if response.status_code == 200:
        resultado = response.json()
        print("✅ Análise via API:\n")
        print(resultado.get("content", resultado))
    else:
        print(f"❌ Erro: {response.text}")
else:
    print("⚠️ Pulando teste via API (não disponível)")


🔍 Analisando via API...

❌ Erro: {"detail":"Not Found"}


## 5. Resumo - OWASP Top 10

O **Security Copilot** detecta vulnerabilidades do OWASP Top 10:

| Código | Vulnerabilidade |
|--------|-----------------|
| A01 | Broken Access Control |
| A02 | Cryptographic Failures |
| A03 | Injection (SQL, XSS, Command) |
| A04 | Insecure Design |
| A05 | Security Misconfiguration |
| A06 | Vulnerable Components |
| A07 | Auth Failures |
| A08 | Data Integrity Failures |
| A09 | Logging Failures |
| A10 | SSRF |

### Métodos Disponíveis

| Método | Descrição |
|--------|-----------|
| `vulnerability_scan()` | Scan completo de vulnerabilidades |
| `analyze_authentication()` | Análise de autenticação |
| `check_data_exposure()` | Verifica exposição de dados |
| `scan_dependencies()` | Analisa dependências vulneráveis |


In [9]:
print("🎉 Demo concluída!")
print("\n🛡️ Dicas de Segurança:")
print("  1. Sempre use queries parametrizadas (evite SQL Injection)")
print("  2. Escape output HTML (evite XSS)")
print("  3. Use bcrypt/argon2 para senhas (não MD5/SHA1)")
print("  4. Nunca exponha secrets no código")
print("  5. Valide e sanitize TODOS os inputs do usuário")


🎉 Demo concluída!

🛡️ Dicas de Segurança:
  1. Sempre use queries parametrizadas (evite SQL Injection)
  2. Escape output HTML (evite XSS)
  3. Use bcrypt/argon2 para senhas (não MD5/SHA1)
  4. Nunca exponha secrets no código
  5. Valide e sanitize TODOS os inputs do usuário
